In [1]:
from model.network.gaitset import SetNet
import torch.optim as optim
import torch.nn as nn
import torch
import os.path as osp
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap.umap_ as umap

g:\Softwares\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder = SetNet(256)
encoder = nn.DataParallel(encoder)
encoder.cuda()

model_name = 'GaitSet'
encoder.load_state_dict(torch.load('C:/Users/TUF/Documents/gait/GaitSet-master/work/checkpoint/GaitSet/GaitSet_CASIA-B_73_False_256_0.2_128_full_30-108100-encoder.ptm'))

<All keys matched successfully>

In [3]:
encoder.eval()

DataParallel(
  (module): SetNet(
    (set_layer1): SetBlock(
      (forward_block): BasicConv2d(
        (conv): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
      )
    )
    (set_layer2): SetBlock(
      (forward_block): BasicConv2d(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (set_layer3): SetBlock(
      (forward_block): BasicConv2d(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (set_layer4): SetBlock(
      (forward_block): BasicConv2d(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (pool2d): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (set_layer5): SetBlock(
      (forward_block): BasicConv2d(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (set_layer6

In [4]:
from model.initialization import initialize_data
from config import conf

train_source, test_source = initialize_data(conf, train = False, test = True)

Initializing data source...
Loading test data...
Data initialization complete.


In [5]:
from copy import deepcopy
import numpy as np

# train_source, test_source = initialize_data(conf, train = False, test = False)
data_config = conf['data']
model_config = conf['model']
model_param = deepcopy(model_config)
model_param['train_source'] = None
model_param['test_source'] = test_source
model_param['train_pid_num'] = data_config['pid_num']
batch_size = int(np.prod(model_config['batch_size']))
model_param['save_name'] = '_'.join(map(str,[
    model_config['model_name'],
    data_config['dataset'],
    data_config['pid_num'],
    data_config['pid_shuffle'],
    model_config['hidden_dim'],
    model_config['margin'],
    batch_size,
    model_config['hard_or_full_trip'],
    model_config['frame_num'],
]))
restore_iter = 108100

In [6]:
import torch.utils.data as tordata
from model.model import Model

model = Model(**model_param)
data_loader = tordata.DataLoader(
            dataset=test_source,
            batch_size=1,
            sampler=tordata.sampler.SequentialSampler(test_source),
            collate_fn=model.collate_fn,
            num_workers=model.num_workers)

In [7]:
feature_list = list()
view_list = list()
seq_type_list = list()
label_list = list()
for i, data in enumerate(data_loader):
    seq, view, seq_type, label, batch_frame = data

    seq[0] = model.np2var(seq[0])
    batch_frame = model.np2var(batch_frame)

    feature, _ = encoder(*seq, batch_frame)
    feature_list.append(feature.view(1, -1).data.cpu().numpy())
    view_list += view
    seq_type_list += seq_type
    label_list += label

In [8]:
data = np.concatenate(feature_list, 0), view_list, seq_type_list, label_list
feature, view, seq_type, label = data
label = np.array(label)
view_list = list(set(view))
view_list.sort()
view_num = len(view_list)
sample_num = len(feature)

In [9]:
from PIL import Image
from torchvision import transforms
import os

transform = transforms.ToTensor()

In [11]:
folder_path = 'data/125'
seqs = os.listdir(folder_path)

for seq in seqs:
    seq_views = os.listdir(folder_path + '/' + str(seq))
    for seq_view in seq_views:
        img_array = []

        for i in range(1, 31):
            img_path = f'{folder_path}/{seq}/{seq_view}/125-{seq}-{seq_view}-{i:03d}.png'
            img = Image.open(img_path).convert('L')
            img = img.resize((44, 64))
            img_array.append(img)
        img_array = torch.stack([transform(img) for img in img_array]).cuda()
        img_array = img_array.view(1, 30, 44, 64)

        features_pack = encoder.module(img_array)
        features_pack[0].view(-1).shape
        feature = np.append(feature, (features_pack[0].view(1, -1).data.cpu().numpy()), axis=0)
        seq_type = np.append(seq_type, [seq], axis=0)
        view = np.append(view, [seq_view], axis=0)
        label = np.append(label, [125], axis=0)
        

In [12]:
len(feature), len(feature[0]), len(view), len(seq_type), len(label)

(895, 15872, 895, 895, 895)

In [13]:
def euclid_dist(a, b):
    a2 = np.sum(a**2, axis=1, keepdims=True)
    b2 = np.sum(b**2, axis=1, keepdims=True)
    dist = a2 + b2.T - 2 * a @ b.T
    dist = np.sqrt(np.maximum(0, dist))
    return dist
    

In [20]:
for i in range(880, len(feature)):
    data = np.reshape(feature[i], (1, -1))
    dist = euclid_dist(feature, data)
    predict = label[np.argsort(dist, axis=0)[1:11]] # Bỏ qua 0 vì là chính nó nên dist = 0
    predict = np.reshape(predict, (-1))
    print('Predict for', seq_type[i], view[i], ':', predict)

Predict for bg-01 000 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for bg-01 090 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for bg-01 180 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for bg-02 054 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for bg-02 144 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for cl-01 000 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for cl-01 090 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for cl-01 180 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for cl-02 054 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for cl-02 144 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for nm-01 000 : ['125' '125' '125' '125' '125' '125' '125' '125' '125' '125']
Predict for nm-01 180 : ['125' '125' '125' '125' '125'